In [14]:
# Problem statement
# 14 controls
# 1 target
# 5 auxilliary 

# b0-13, t, a0-4

# Design 1: (3-4-3) 
# 4 3-bit in place control gates (b0-11)->(a0-3)
# 1 4-bit in place control gate taking 2 remaining bits and 2 auxiliary bits (b12-13, a0-1) -> a4
# 3 3-bit in place control gate that takes (a2-4) -> t


In [15]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy

# import basic plot tools
from qiskit.visualization import plot_histogram 

In [16]:
control_qubits = QuantumRegister(14, name="c")
target_qubit = QuantumRegister(1, name='t')
ancilla_qubits = QuantumRegister(5, name='a')
classical_bits = ClassicalRegister(1, name='out')
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)

#qc.mcx()

In [17]:
# no optimization - design 0
qc.mcx(control_qubits, target_qubit[0], ancilla_qubits=ancilla_qubits, mode='recursion')
qc.measure(target_qubit[0], classical_bits[0])

In [5]:
qc.draw()

c_0: ──■──────
         │      
  c_1: ──■──────
         │      
  c_2: ──■──────
         │      
  c_3: ──■──────
         │      
  c_4: ──■──────
         │      
  c_5: ──■──────
         │      
  c_6: ──■──────
         │      
  c_7: ──■──────
         │      
  c_8: ──■──────
         │      
  c_9: ──■──────
         │      
 c_10: ──■──────
         │      
 c_11: ──■──────
         │      
 c_12: ──■──────
         │      
 c_13: ──■──────
       ┌─┴──┐┌─┐
    t: ┤0   ├┤M├
       │  X │└╥┘
  a_0: ┤1   ├─╫─
       └────┘ ║ 
  a_1: ───────╫─
              ║ 
  a_2: ───────╫─
              ║ 
  a_3: ───────╫─
              ║ 
  a_4: ───────╫─
              ║ 
out: 1/═══════╩═
              0

In [18]:
aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

890
OrderedDict([('u', 712), ('cx', 656), ('measure', 1)])


In [19]:
# design 1 
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
for i in range(14):
    qc.x(control_qubits[i])

for i in range(4):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])

qc.mcx(control_qubits[12:14] + ancilla_qubits[0:2], ancilla_qubits[4], mode='recursion')
qc.mcx(ancilla_qubits[2:5], target_qubit[0], mode='recursion')
qc.measure(target_qubit[0], classical_bits[0])


In [24]:
# design 2 
qc = QuantumCircuit(control_qubits, target_qubit, ancilla_qubits, classical_bits)
for i in range(14):
    qc.x(control_qubits[i])

for i in range(3):
    qc.mcx(control_qubits[3*i:3*(i+1)], ancilla_qubits[i])
# 0 - 8
qc.mcx(control_qubits[9:13], ancilla_qubits[3])

qc.mcx(ancilla_qubits[0:2] + control_qubits[13:14], ancilla_qubits=ancilla_qubits[4], mode='recursion')

qc.mcx(ancilla_qubits[2:5], target_qubit[0])
qc.measure(target_qubit[0], classical_bits[0])

TypeError: mcx() missing 1 required positional argument: 'target_qubit'

In [9]:
qc.draw()

┌───┐     ┌───┐                                                »
  c_0: ──■───┤ X ├──■──┤ X ├──────────────────────■─────────────────────────»
         │   ├───┤  │  ├───┤                      │                         »
  c_1: ──■───┤ X ├──■──┤ X ├──────────────────────■─────────────────────────»
         │   ├───┤  │  ├───┤                      │                         »
  c_2: ──■───┤ X ├──■──┤ X ├──────────────────────■─────────────────────────»
         │   ├───┤  │  └───┘┌───┐                 │                         »
  c_3: ──■───┤ X ├──┼────■──┤ X ├─────────────────┼────■────────────────────»
         │   ├───┤  │    │  ├───┤                 │    │                    »
  c_4: ──■───┤ X ├──┼────■──┤ X ├─────────────────┼────■────────────────────»
         │   ├───┤  │    │  ├───┤                 │    │                    »
  c_5: ──■───┤ X ├──┼────■──┤ X ├─────────────────┼────■────────────────────»
         │   ├───┤  │    │  └───┘┌───┐            │    │                    »
  c_6: ──■───┤ X ├──┼────┼────■──┤ X ├────────────┼────┼───────■────────────»
         │   ├───┤  │    │    │  ├───┤            │    │       │            »
  c_7: ──■───┤ X ├──┼────┼────■──┤ X ├────────────┼────┼───────■────────────»
         │   ├───┤  │    │    │  ├───┤            │    │       │            »
  c_8: ──■───┤ X ├──┼────┼────■──┤ X ├────────────┼────┼───────■────────────»
         │   ├───┤  │    │    │  └───┘┌───┐       │    │       │            »
  c_9: ──■───┤ X ├──┼────┼────┼────■──┤ X ├───────┼────┼───────┼────■───────»
         │   ├───┤  │    │    │    │  ├───┤       │    │       │    │       »
 c_10: ──■───┤ X ├──┼────┼────┼────■──┤ X ├───────┼────┼───────┼────■───────»
         │   ├───┤  │    │    │    │  ├───┤       │    │       │    │       »
 c_11: ──■───┤ X ├──┼────┼────┼────■──┤ X ├───────┼────┼───────┼────■───────»
         │   ├───┤  │    │    │    │  └───┘┌───┐  │    │       │    │       »
 c_12: ──■───┤ X ├──┼────┼────┼────┼────■──┤ X ├──┼────┼───────┼────■───────»
         │   ├───┤  │    │    │    │    │  ├───┤  │    │       │    │       »
 c_13: ──■───┤ X ├──┼────┼────┼────┼────■──┤ X ├──┼────┼───────┼────┼────■──»
       ┌─┴──┐└┬─┬┘  │    │    │    │    │  ├───┤  │    │  ┌─┐  │    │    │  »
    t: ┤0   ├─┤M├───┼────┼────┼────┼────┼──┤ X ├──┼────┼──┤M├──┼────┼────┼──»
       │  X │ └╥┘ ┌─┴─┐  │    │    │    │  └─┬─┘┌─┴─┐  │  └╥┘  │    │    │  »
  a_0: ┤1   ├──╫──┤ X ├──┼────┼────┼────■────┼──┤ X ├──┼───╫───┼────┼────■──»
       └────┘  ║  └───┘┌─┴─┐  │    │    │    │  └───┘┌─┴─┐ ║   │    │    │  »
  a_1: ────────╫───────┤ X ├──┼────┼────■────┼───────┤ X ├─╫───┼────┼────■──»
               ║       └───┘┌─┴─┐  │    │    │       └───┘ ║ ┌─┴─┐  │    │  »
  a_2: ────────╫────────────┤ X ├──┼────┼────■─────────────╫─┤ X ├──┼────┼──»
               ║            └───┘┌─┴─┐  │    │             ║ └───┘┌─┴─┐  │  »
  a_3: ────────╫─────────────────┤ X ├──┼────■─────────────╫──────┤ X ├──┼──»
               ║                 └───┘┌─┴─┐  │             ║      └───┘┌─┴─┐»
  a_4: ────────╫──────────────────────┤ X ├──■─────────────╫───────────┤ X ├»
               ║                      └───┘                ║           └───┘»
out: 1/════════╩═══════════════════════════════════════════╩════════════════»
               0                                           0                »
«               
«  c_0: ────────
«               
«  c_1: ────────
«               
«  c_2: ────────
«               
«  c_3: ────────
«               
«  c_4: ────────
«               
«  c_5: ────────
«               
«  c_6: ────────
«               
«  c_7: ────────
«               
«  c_8: ────────
«               
«  c_9: ────────
«               
« c_10: ────────
«               
« c_11: ────────
«               
« c_12: ────────
«               
« c_13: ────────
«       ┌───┐┌─┐
«    t: ┤ X ├┤M├
«       └─┬─┘└╥┘
«  a_0: ──┼───╫─
«         │   ║ 
«  a_1: ──┼───╫─
«         │   ║ 
«  a_2: ──■───╫─
«         │   ║ 
«  a_3: ──■───╫─
«         │  

In [27]:
# aer_simulator = Aer.get_backend('aer_simulator')
qc_transpiled = transpile(qc, aer_simulator, basis_gates=['u', 'cx'])
# qobj = assemble(qc_transpiled)
# result = aer_simulator.run(qobj).result()
# print(result.get_counts())

In [11]:
qc_transpiled.draw()

global phase: 1.5626e-14
                                                                         »
  c_0: ──────────────────────────────────────────────────────────────────»
                                                                         »
  c_1: ──────────────────────────────────────────────────────────────────»
                                                                         »
  c_2: ──────────────────────────────────────────────────────────────────»
         ┌────────────┐                             ┌───────────────┐    »
  c_3: ──┤ U(0,0,π/4) ├───■───────────────────■─────┤ U(π/2,π/4,-π) ├────»
         └────────────┘   │                   │     └───────────────┘    »
  c_4: ───────────────────┼───────────────────┼──────────────────────────»
                          │                   │                          »
  c_5: ───────────────────┼───────────────────┼──────────────────────────»
                          │                   │                          »
  c_6: ───────────────────┼───────────────────┼──────────────────────────»
         ┌────────────┐ ┌─┴─┐┌─────────────┐┌─┴─┐┌──────────────────────┐»
  c_7: ──┤ U(π/2,0,π) ├─┤ X ├┤ U(0,0,-π/4) ├┤ X ├┤ U(0,1.4066,-0.62118) ├»
         └────────────┘ └───┘└─────────────┘└───┘└──────────────────────┘»
  c_8: ──────────────────────────────────────────────────────────────────»
                                                                         »
  c_9: ──────────────────────────────────────────────────────────────────»
         ┌────────────┐                                                  »
 c_10: ──┤ U(0,0,π/4) ├──────────────────────────────────────────────────»
         ├────────────┤                                                  »
 c_11: ──┤ U(0,0,π/8) ├───■───────────────────■──────────────────────────»
         ├────────────┤ ┌─┴─┐┌─────────────┐┌─┴─┐                        »
 c_12: ──┤ U(0,0,π/8) ├─┤ X ├┤ U(0,0,-π/8) ├┤ X ├───────────■────────────»
         ├────────────┤ └───┘└─────────────┘└───┘         ┌─┴─┐          »
 c_13: ──┤ U(0,0,π/8) ├───────────────────────────────────┤ X ├──────────»
       ┌─┴────────────┴┐                                  └───┘          »
    t: ┤ U(π/2,π/8,-π) ├─────────────────────────────────────────────────»
       └─┬────────────┬┘                                                 »
  a_0: ──┤ U(π/2,0,π) ├──────────────────────────────────────────────────»
       ┌─┴────────────┴┐                                                 »
  a_1: ┤ U(π/2,π/8,-π) ├─────────────────────────────────────────────────»
       ├───────────────┤                                                 »
  a_2: ┤ U(π/2,π/8,-π) ├─────────────────────────────────────────────────»
       ├───────────────┤                                                 »
  a_3: ┤ U(π/2,π/8,-π) ├─────────────────────────────────────────────────»
       └─┬────────────┬┘                                                 »
  a_4: ──┤ U(π/2,0,π) ├──────────────────────────────────────────────────»
         └────────────┘                                                  »
out: 1/══════════════════════════════════════════════════════════════════»
                                                                         »
«                                                                   »
«  c_0: ──────────────────────────────────────■─────────────────────»
«                                             │                     »
«  c_1: ──────────────────────────────────────┼─────────────────────»
«                                             │                     »
«  c_2: ───────■──────────────────────────────┼─────────────────────»
«            ┌─┴─┐     ┌───────────────┐    ┌─┴─┐     ┌────────────┐»
«  c_3: ─────┤ X ├─────┤ U(π/2,0,3π/4) ├────┤ X ├─────┤ U(0,0,π/4) ├»
«            └───┘     └───────────────┘    └───┘     └────────────┘»
«  c_4: ────────────────────────────────────────────────────────────»
«                                                 

In [28]:
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

65
OrderedDict([('u', 93), ('cx', 78)])


In [23]:
print(qc_transpiled.depth())
print(qc_transpiled.count_ops())

92
OrderedDict([('u', 119), ('cx', 106), ('measure', 1)])
